### Fill out the next cell with required values

**This example expects a CSV file from Snowflake with the following format (this example assumes a database name of "TYA_DATA", schema "PUBLIC", and table "PEOPLE")**

```
TAG_DATABASE,TAG_SCHEMA,OBJECT_NAME,COLUMN_NAME,TAG_VALUE
TYA_DATA,PUBLIC,PEOPLE,NAME_LAST,PERSONNAME
TYA_DATA,PUBLIC,PEOPLE,ADDRESS,ADDRESS

```

**In Snowflake, we ran the following SQL and then waited two hours (a standard Snowflake timing/cache item):**

```
create or replace tag satori_c12n_pii allowed_values 
'ADDRESS' , 'EMAIL', 'PERSONNAME', 'PHONE', 'SSN';

create or replace tag satori_c12n_piilinkable allowed_values 
'GENDER', 'USERNAME', 'CITY', 'STATE', 'ZIPCODE', 'STATE';

ALTER TABLE tya_data.public.people
  MODIFY COLUMN address
   SET TAG satori_c12n_pii ='ADDRESS';

ALTER TABLE tya_data.public.people
  MODIFY COLUMN email
   SET TAG satori_c12n_pii ='EMAIL';

ALTER TABLE tya_data.public.people
  MODIFY COLUMN name_last
   SET TAG satori_c12n_pii ='PERSONNAME';
```

**After two hours, we ran the following sql and then downloaded the results as a CSV file**
```
SELECT tag_database, tag_schema, object_name, column_name, tag_value FROM
SNOWFLAKE.ACCOUNT_USAGE.TAG_REFERENCES
WHERE TAG_NAME = 'SATORI_C12N_PII';
```

**This file is what this Jupyter Notebook expects to see**

In [ ]:
# your account info, following must be filled in
satori_account_id = ""
satori_serviceaccount_id = ""
satori_serviceaccount_key = ""

# leave this host value as is for Satori production accounts
apihost = "app.satoricyber.com"

# location of our csv input file
csv_input_file = "tags_for_locations_from_snowflake.csv"

# action to take, the choices are 'add' or 'remove' - this allows for a full UNDO of the csv file
action = "add"

# this message will appear in the location history for each field that is updated
location_history_message = "Location Updated by Snowflake CSV Import Process, action: " + action

# The Snowflake database "Display Name" connection name from Satori Datastores
snowflake_datastore_name = "New YCB Snowflake Connection"

#### for demonstration purposes, no changes needed below this line

In [ ]:
import json
import requests
import csv
import io

In [ ]:
# our auth function to get a bearer token

def satori_auth(satori_serviceaccount_id, satori_serviceaccount_key, apihost):
    auth_headers = {'content-type': 'application/json','accept': 'application/json'}
    auth_url = "https://{}/api/authentication/token".format(apihost)
    auth_body = json.dumps(
    {
        "serviceAccountId": satori_serviceaccount_id,
        "serviceAccountKey": satori_serviceaccount_key
    })
    try:
        r = requests.post(auth_url, headers=auth_headers, data=auth_body)
        response = r.json()
        satori_token = response["token"]
    except Exception as err:
        print("Bearer Token Failure: :", err)
        print("Exception TYPE:", type(err))
    else:
        return satori_token

In [ ]:
# get the token and also create request header

session_token = satori_auth(satori_serviceaccount_id, satori_serviceaccount_key, apihost)
#example
#print(session_token)

headers = {'Authorization': 'Bearer {}'.format(session_token), 'Content-Type': 'application/json', 'Accept': 'application/json'}


In [ ]:
# get a Datastore ID by searching for its name

def get_datastore_id(datastore_name):

    url = "https://{}/api/v1/datastore?accountId={}&search={}".format(apihost, 
                                                                    satori_account_id, 
                                                                    datastore_name)
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as err:
        print("EXCEPTION: ", type(err))
    else:
        return response.json()['records'][0]['id']
        
#example
#print(get_datastore_id("AWS Postgres Data"))

In [ ]:
# get a Location ID by searching within datastore_id for database.schema.table.field.semiPath

def get_location_id(datastore_id, location_prefix):

    url = "https://{}/api/locations/{}/query?pageSize=100&dataStoreId={}&locationPrefix={}".format(
                                                                    apihost, 
                                                                    satori_account_id, 
                                                                    datastore_id,
                                                                    location_prefix)
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as err:
        print("EXCEPTION: ", type(err))
    else:
        return response.json()['records'][0]['id']
        
#example
#print(get_location_id('b35585f4-08b4-415c-82a1-d6c1af17fcbb', 'postgres.public.v2_people.blood_type'))

In [ ]:
# load all of the custom taxonomy as a dictionary

def load_all_custom_taxonomy():
                
    url = "https://{}/api/v1/taxonomy/custom?accountId={}".format(apihost,satori_account_id)
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as err:
        print("EXCEPTION: ", type(err))
    else:
        custom_taxonomy = {}
        for item in response.json()['records']:
            custom_taxonomy[item['tag']] = item['id']
        return custom_taxonomy

#example    
#print(load_all_custom_taxonomy())

#now actually do it
custom_taxonomy = load_all_custom_taxonomy()

In [ ]:
# build our PUT payload, takes into account whether we are adding or removing the tags

def build_update_location_payload(action, taxonomy_id):
    payload =''
    if action == 'add':
        payload = json.dumps(
            {
              "addTags": [
                taxonomy_id
              ],
              "removeTags": [
              ],
              "notes": location_history_message
            }    
        )
    elif action == 'remove':
        payload = json.dumps(
            {
              "addTags": [
              ],
              "removeTags": [
                taxonomy_id
              ],
              "notes": location_history_message
            }    
        )
    return payload
    
#example
#print(build_update_location_payload("51d3d892-26f6-41b8-ab0c-c5638c386379"))

In [ ]:
# update one location with one tag (aka taxonomy id)

def update_one_location(action, location_id, custom_taxonomy_id):
    url = "https://{}/api/locations/{}".format(apihost,location_id)
    
    payload = build_update_location_payload(action, custom_taxonomy_id)
    
    try:
        response = requests.put(url=url, headers=headers, data=payload)
        response.raise_for_status()
    except requests.exceptions.RequestException as err:
        print(str(err))
    else:
        return response.json()

In [ ]:
# load our input from the csv file

tags_for_locations_from_csv_file = []

with open(csv_input_file, 'r', newline='\n') as csvfile:
    next(csvfile)
    csvreader = csv.reader(csvfile, delimiter=',', quotechar='"')
    for row in csvreader:
        tags_for_locations_from_csv_file.append(row)

In [ ]:
for item in tags_for_locations_from_csv_file:
    
    location_prefix = item[0] + "." + item[1] + "." + item[2] + "." + item[3]
    taxonomy_id = item[4]
    
    datastore_id = get_datastore_id(snowflake_datastore_name)
    location_id = get_location_id(datastore_id, location_prefix)
    response = update_one_location(action, location_id, taxonomy_id)
    print("ACTION TAKEN: " + str(response) + "\n")
